[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2021/blob/main/40--spark/01--rdd.ipynb)

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz -O spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

In [ ]:
!pip install -q pyspark findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Test it")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

21/12/11 07:05:03 WARN Utils: Your hostname, Jakubs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.8.8 instead (on interface en0)
21/12/11 07:05:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/11 07:05:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [6]:
sc = spark.sparkContext

### Resilient Distributed Dataset or RDD

An RDD is a distributed collection of elements. All work in Spark is expressed as either creating new RDDs, transforming existing RDDs, or calling actions on RDDs to compute a result. Spark automatically distributes the data contained in RDDs across your cluster and parallelizes the operations you perform on them.

https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=flatmap#pyspark.RDD.cache

#### Creating and RDD using parallelize
Another way of creating an RDD is to parallelize an already existing list.

In [7]:
A = ((a, a*a) for a in range(1000))

In [8]:
data = sc.parallelize(A)

In [9]:
data.count()

1000

In [10]:
data.take(10)

[(0, 0),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (9, 81)]

#### Creating a RDD from a file
The most common way of creating an RDD is to load it from a file. Notice that Spark's textFile can handle compressed files directly.

In [23]:
!mkdir -p var
!wget https://wolnelektury.pl/media/book/txt/krzyzacy-tom-pierwszy.txt -O var/krzyzacy-1.txt

--2021-12-11 07:12:08--  https://wolnelektury.pl/media/book/txt/krzyzacy-tom-pierwszy.txt
Resolving wolnelektury.pl (wolnelektury.pl)... 51.83.143.148
Connecting to wolnelektury.pl (wolnelektury.pl)|51.83.143.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 740054 (723K) [text/plain]
Saving to: ‘var/krzyzacy-1.txt’

var/krzyzacy-1.txt  100%[===================>] 722.71K  --.-KB/s    in 0.07s   

2021-12-11 07:12:08 (10.5 MB/s) - ‘var/krzyzacy-1.txt’ saved [740054/740054]



In [24]:
file = sc.textFile('var/krzyzacy-1.txt')

In [25]:
file.count()

7607

In [26]:
file.take(10)

['Henryk Sienkiewicz',
 '',
 'Krzyżacy',
 'Tom I',
 '',
 'ISBN 978-83-288-2813-1',
 '',
 '',
 '',
 '']

In [27]:
filtered = file.filter(lambda x: x != '')

In [28]:
filtered.take(10)

['Henryk Sienkiewicz',
 'Krzyżacy',
 'Tom I',
 'ISBN 978-83-288-2813-1',
 'Rozdział pierwszy',
 'W Tyńcu, w gospodzie „Pod Lutym Turem”, należącej do opactwa, siedziało kilku ludzi, słuchając opowiadania wojaka bywalca, który z dalekich stron przybywszy, prawił im o przygodach, jakich na wojnie i w czasie podróży doznał. Człek był brodaty, w sile wieku, pleczysty, prawie ogromny, ale wychudły; włosy nosił ujęte w pątlik, czyli w siatkę naszywaną paciorkami; na sobie miał skórzany kubrak z pręgami wyciśniętymi przez pancerz, na nim pas, cały z miedzianych klamr; za pasem nóż w rogowej pochwie, przy boku zaś krótki kord podróżny.',
 'Tuż przy nim za stołem siedział młodzieńczyk o długich włosach i wesołym spojrzeniu, widocznie jego towarzysz lub może giermek, bo przybrany także po podróżnemu, w taki sam powyciskany od zbroicy skórzany kubrak. Resztę towarzystwa stanowiło dwóch ziemian z okolic Krakowa i trzech mieszczan w czerwonych składanych czapkach, których cienkie końce zwieszały si

In [29]:
counts = filtered \
    .flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda acc, item: acc + item)


In [30]:
counts.takeOrdered(5)

[('', 24),
 ('(bo', 1),
 ('(http://creativecommons.org/licenses/by-sa/3.0/).', 1),
 ('(http://wolnelektury.pl).', 1),
 ('(precz', 1)]

In [31]:
counts.takeOrdered(5, key=lambda x: -x[1])

[('—', 4339), ('i', 3906), ('się', 2486), ('nie', 2024), ('w', 2017)]

In [32]:
!mkdir -p var
!wget https://wolnelektury.pl/media/book/txt/krzyzacy-tom-drugi.txt -O var/krzyzacy-2.txt

--2021-12-11 07:13:17--  https://wolnelektury.pl/media/book/txt/krzyzacy-tom-drugi.txt
Resolving wolnelektury.pl (wolnelektury.pl)... 51.83.143.148
Connecting to wolnelektury.pl (wolnelektury.pl)|51.83.143.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 840889 (821K) [text/plain]
Saving to: ‘var/krzyzacy-2.txt’

var/krzyzacy-2.txt  100%[===================>] 821.18K  --.-KB/s    in 0.1s    

2021-12-11 07:13:17 (8.38 MB/s) - ‘var/krzyzacy-2.txt’ saved [840889/840889]



In [33]:
file = sc.textFile('var/krzyzacy*')

In [34]:
file.count()

15477